# Cálculo de derivadas presión atmosférica y graficación

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

**Librerías**

In [1]:
import pandas as pd
import numpy as np
import datetime
import statistics
import glob
import os
import csv
import re
import gc
from collections import deque
from datetime import timedelta
from scipy import stats
from openpyxl import Workbook
from openpyxl.chart import LineChart, Reference
from openpyxl.chart import BarChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows

____

### Pruebas unitarias

#### Datos con QC

In [3]:
df_example = pd.read_csv('../../OE_3_QC_Variables/1_PresionAtmosferica/QCResult_Patm/V3/Estacion_0015015060_qc.csv', encoding='latin-1', dtype={'Estado_Anterior':str})

In [3]:
## Derivadas diarias, mensuales y anuales
# Presión atmosférica media diaria
def PA_60_MEDIA_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
    df.reset_index(inplace=True)
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
    # Se filtran los que hayan superaddo las pruebas
    dfC = df[df['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PC']]))]

    # Ajustar la hora de cada registro para que corresponda al rango deseado
    dfC[columna_fecha] = dfC[columna_fecha] - pd.Timedelta(hours=1)
    
    # Establecer la columna de fecha como índice
    dfC.set_index(columna_fecha, inplace=True)
    
    # Calcular el total de registros esperados por día pluviométrico
    total_esperado_por_dia = 24

    # Función para verificar si un día pluviométrico tiene suficientes datos
    def complet_dia(sub_df):
        return len(sub_df) >= total_esperado_por_dia * porc_min

    # Filtrar los días con suficientes datos y calcular el promedio diario
    df_filtrado = dfC.groupby([dfC.index.date]).filter(complet_dia)
    PA_60_med_d = df_filtrado[[columna_valor]].resample('D').mean()

    return PA_60_med_d

# Ejemplo de uso de la función
dfmn_med_d = PA_60_MEDIA_D(df_example)

# Presión atmosférica media mensual
def PA_60_MEDIA_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):    
    df.reset_index(inplace=True)
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    
    # Establecer la columna 'Fecha' como índice
    df.set_index(columna_fecha, inplace=True)
    
    days_in_month = df.index.to_series().dt.days_in_month
    days_in_month = days_in_month.resample('M').first()
    
    # Función para verificar si una hora específica tiene suficientes datos
    def complet_mes(sub_df):
        mes = sub_df.index[0].month
        total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
        return len(sub_df) >= total_esperado * porc_min

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
    PA_60_med_m = df_filtrado[['Valor']].resample('M').mean()
    
    return PA_60_med_m

dfmn_med_m = PA_60_MEDIA_M(dfmn_med_d)

# Presión atmosférica del Aire a 2 metros media anual
def PA_60_MEDIA_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
    df.reset_index(inplace=True)
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    
    # Función para verificar si una hora específica tiene suficientes datos
    def complet_anio(sub_df):
        return len(sub_df) >= 12 * porc_min
       
    # Antes de resample, establecer la columna 'Fecha' como índice
    df.set_index(columna_fecha, inplace=True)

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
    PA_60_med_a = df_filtrado[['Valor']].resample('Y').mean()
    
    return PA_60_med_a

dfmn_med_a = PA_60_MEDIA_A(dfmn_med_m)

C:\Users\user\AppData\Local\Temp\ipykernel_18208\1903284964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfC[columna_fecha] = dfC[columna_fecha] - pd.Timedelta(hours=1)


In [4]:
## -- Gráficas y export en excel
# Se crea un nuevo archivo Excel con openpyxl
wb = Workbook()
sheets = {
    'PA_60_MEDIA_D': dfmn_med_d, 
    'PA_60_MEDIA_M': dfmn_med_m,
    'PA_60_MEDIA_A': dfmn_med_a
}

# Si el workbook todavía tiene la hoja por defecto, se elimina
if "Sheet" in wb.sheetnames:
    del wb["Sheet"]

for sheet_name, data in sheets.items():
    ws = wb.create_sheet(title=sheet_name)
    
    # Agregamos los datos al Excel
    for r_idx, row in enumerate(dataframe_to_rows(data, index=True, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)
    
    # Crear una gráfica
    chart = LineChart()
    chart.title = sheet_name
    chart.style = 5
    chart.y_axis.title = 'Presión atmosf. (hPa)'
    chart.x_axis.title = 'Fecha'
    
    # Establecer datos para la gráfica
    max_row = ws.max_row
    values = Reference(ws, min_col=2, min_row=2, max_col=2, max_row=max_row)
    dates = Reference(ws, min_col=1, min_row=3, max_col=1, max_row=max_row)
    chart.add_data(values, titles_from_data=True)
    chart.set_categories(dates)
    
    # Quitar la leyenda
    chart.legend = None
        
    # Posicionar la gráfica en el Excel
    ws.add_chart(chart, "E3")

# Guardar el archivo Excel
wb.save("Agreg_graf_PA_60_AUT_60_QC_V1.xlsx")

## Función para cálculo masivo de derivadas y generación de gráficas

In [3]:
def calc_deriv_patm(carpeta, chunk_size=540000):
    archivos = os.listdir(carpeta)

    # Se recorre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
        
            # Se procesan los archivos csv por fragmentos
            reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=chunk_size)
            
            for chunk in reader:
                # Se generan dataframes analizados
                # De cada chunk se transforma a datetime la serie/columna 'Fecha'
                try:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                except ValueError:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S')
                    
                try:
                    dfC = chunk[~chunk['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO','0PAT','0PER']]))] # Se comenta si es de crudos
                    station_value = dfC['Station'].values[0] #chunk['Station'].values[0]   
                except IndexError:
                    print(f"Error en el archivo {archivo}: dfC está vacío. Saltando al siguiente archivo.")
                    continue  # Sale del bucle de chunks y continúa con el siguiente archivo

                # Presión atmosférica media diaria
                def PA_60_MEDIA_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    df.reset_index(inplace=True)
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                    # Ajustar la hora de cada registro para que corresponda al rango deseado
                    df[columna_fecha] = df[columna_fecha] - pd.Timedelta(hours=1)
                    
                    # Establecer la columna de fecha como índice
                    df.set_index(columna_fecha, inplace=True)
                    
                    # Calcular el total de registros esperados por día pluviométrico
                    total_esperado_por_dia = 24
                
                    # Función para verificar si un día pluviométrico tiene suficientes datos
                    def complet_dia(sub_df):
                        return len(sub_df) >= total_esperado_por_dia * porc_min
                
                    # Filtrar los días con suficientes datos y calcular el promedio diario
                    df_filtrado = df.groupby([df.index.date]).filter(complet_dia)
                    pa_60_media_d = df_filtrado[[columna_valor]].resample('D').mean()
                    
                    return pa_60_media_d
                
                # Ejemplo de uso de la función
                dfpa_med_d = PA_60_MEDIA_D(dfC) #chunk si es de datos crudos #dfC si son datos con QC 
                
                # Presión atmosférica del Aire a 2 metros media mensual
                dfpa_med_d_c = dfpa_med_d.copy()
                def PA_60_MEDIA_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):    
                    df.reset_index(inplace=True)
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    
                    # Establecer la columna 'Fecha' como índice
                    df.set_index(columna_fecha, inplace=True)
                    
                    days_in_month = df.index.to_series().dt.days_in_month
                    days_in_month = days_in_month.resample('M').first()
                    
                    # Función para verificar si una hora específica tiene suficientes datos
                    def complet_mes(sub_df):
                        mes = sub_df.index[0].month
                        total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
                        return len(sub_df) >= total_esperado * porc_min
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
                    pa_60_media_m = df_filtrado[['Valor']].resample('M').mean()
                    
                    return pa_60_media_m
                
                dfpa_med_m = PA_60_MEDIA_M(dfpa_med_d_c)
                
                # Presión atmosférica del Aire a 2 metros media anual
                dfpa_med_m_c = dfpa_med_m.copy()
                def PA_60_MEDIA_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    df.reset_index(inplace=True)
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    
                    # Función para verificar si una hora específica tiene suficientes datos
                    def complet_anio(sub_df):
                        return len(sub_df) >= 12 * porc_min
                       
                    # Antes de resample, establecer la columna 'Fecha' como índice
                    df.set_index(columna_fecha, inplace=True)
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
                    pa_60_media_a = df_filtrado[['Valor']].resample('Y').mean()

                    return pa_60_media_a
                
                dfpa_med_a = PA_60_MEDIA_A(dfpa_med_m_c)
        
                
                # Se crea un nuevo archivo Excel con openpyxl
                wb = Workbook()
                sheets = {
                    'PA_60_MEDIA_D_qc': dfpa_med_d, 
                    'PA_60_MEDIA_M_qc': dfpa_med_m,
                    'PA_60_MEDIA_A_qc': dfpa_med_a
                }
                
                # Si el workbook todavía tiene la hoja por defecto, se elimina
                if "Sheet" in wb.sheetnames:
                    del wb["Sheet"]
                
                for sheet_name, data in sheets.items():
                    ws = wb.create_sheet(title=sheet_name)
                    
                    # Agregamos los datos al Excel
                    for r_idx, row in enumerate(dataframe_to_rows(data, index=True, header=True), 1):
                        for c_idx, value in enumerate(row, 1):
                            ws.cell(row=r_idx, column=c_idx, value=value)
                    
                    # Crear una gráfica
                    chart = LineChart()
                    chart.title = sheet_name
                    chart.style = 5
                    chart.y_axis.title = 'Presión atmosférica (hPa)'
                    chart.x_axis.title = 'Fecha'
                    
                    # Establecer datos para la gráfica
                    max_row = ws.max_row
                    values = Reference(ws, min_col=2, min_row=2, max_col=2, max_row=max_row)
                    dates = Reference(ws, min_col=1, min_row=3, max_col=1, max_row=max_row)
                    chart.add_data(values, titles_from_data=True)
                    chart.set_categories(dates)
                    
                    # Quitar la leyenda
                    chart.legend = None
                        
                    # Posicionar la gráfica en el Excel
                    ws.add_chart(chart, "E3")
                
                # Nombres archivos
                nombre_archivo_salida = os.path.join(carpeta, archivo[:22] + '_deriv.xlsx') #archivo[:19] el original #archivo[:22] con qc
                
                # Guardar el archivo Excel
                wb.save(nombre_archivo_salida)

In [5]:
calc_deriv_patm('../../OE_3_QC_Variables/1_PresionAtmosferica/Test_QC')

C:\Users\palvarez\AppData\Local\Temp\ipykernel_16692\2707177675.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna_fecha] = df[columna_fecha] - pd.Timedelta(hours=1)
C:\Users\palvarez\AppData\Local\Temp\ipykernel_16692\2707177675.py:68: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  days_in_month = days_in_month.resample('M').first()
C:\Users\palvarez\AppData\Local\Temp\ipykernel_16692\2707177675.py:78: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pa_60_media_m = df_filtrado[['Valor']].resample('M').mean()
C:\Users\palvarez\AppData\Local\Temp\ipykernel_16692\2707177675.py:101: FutureWarning: 'Y' is deprecated and will be remo

----

## Función cálculo masivo de derivadas y alistamiento Cassandra

In [3]:
def calc_deriv_PA_60(carpeta):#, chunk_size=540000):
    archivos = os.listdir(carpeta)

    # Se crean carpetas para cada tipo de DataFrame si no existen
    carpetas_salidas = ['PA_60_MEDIA_D_QC', 'PA_60_MEDIA_M_QC', 'PA_60_MEDIA_A_QC']
    for cs in carpetas_salidas:
        os.makedirs(os.path.join(carpeta, cs), exist_ok=True)
    
    # Procesar cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
            df = pd.read_csv(ruta_archivo, encoding='latin-1')

            # Procesar la fecha y filtrar según estado, como en el ejemplo original
            try:
                df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
            except ValueError:
                df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d %H:%M:%S')
                  
            try:
                dfC = df[df['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PC']]))] # Se comenta si es de crudos
                station_value = dfC['Station'].values[0] #df['Station'].values[0]
            except IndexError:
                print(f"Error en el archivo {archivo}: dfC está vacío. Saltando al siguiente archivo.")
                continue  # Sale del bucle de chunks y continúa con el siguiente archivo

            # Presión atmosférica media diaria
            def PA_60_MEDIA_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                if df.empty:
                    return df
                df.reset_index(inplace=True)
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                # Ajustar la hora de cada registro para que corresponda al rango deseado
                df[columna_fecha] = df[columna_fecha] - pd.Timedelta(hours=1)
                # Establecer la columna de fecha como índice
                df.set_index(columna_fecha, inplace=True)
                
                # Calcular el total de registros esperados por día pluviométrico
                total_esperado_por_dia = 24
                # Función para verificar si un día pluviométrico tiene suficientes datos
                def complet_dia(sub_df):
                    return len(sub_df) >= total_esperado_por_dia * porc_min
            
                # Filtrar los días con suficientes datos y calcular el promedio diario
                df_filtrado = df.groupby([df.index.date]).filter(complet_dia)
                # Verificar si el DataFrame filtrado está vacío
                if df_filtrado.empty:
                    print(f"DataFrame vacío después de filtrar por días válidos. Regresando {archivo[:19]} vacío.")
                    return df_filtrado
                    
                # Se calcula la media
                PA_60_med_d = df_filtrado[[columna_valor]].resample('D').mean()

                # Cambio de contenido de columnas
                PA_60_med_d['Station'] = df['Station'].iloc[0]
                PA_60_med_d['Station'] = PA_60_med_d['Station'].astype('int64')
                PA_60_med_d['Sensor'] = 'PA_60_MEDIA_D_QC'
                
                # Reset index
                PA_60_med_d.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['Station', 'Sensor', 'Fecha', 'Valor']
                # Reordenar las columnas usando el nuevo orden
                PA_60_med_d = PA_60_med_d[nuevo_orden]
            
                # Convertir 'Fecha' de nuevo a datetime para uniformidad
                PA_60_med_d['Fecha'] = pd.to_datetime(PA_60_med_d['Fecha'])
            
                return PA_60_med_d

            dfmn_med_d = PA_60_MEDIA_D(dfC)
            
            # Presión atmosférica media mensual
            dfmn_med_d_c = dfmn_med_d.copy()
            def PA_60_MEDIA_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):    
                if df.empty:
                    return df
                df.reset_index(inplace=True)
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                # Establecer la columna 'Fecha' como índice
                df.set_index(columna_fecha, inplace=True)
                
                days_in_month = df.index.to_series().dt.days_in_month
                days_in_month = days_in_month.resample('M').first()
                
                # Función para verificar si una hora específica tiene suficientes datos
                def complet_mes(sub_df):
                    mes = sub_df.index[0].month
                    total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
                    return len(sub_df) >= total_esperado * porc_min
            
                # Luego de establecer el índice, aplicar resample
                df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
                # Verificar si el DataFrame filtrado está vacío
                if df_filtrado.empty:
                    print(f"DataFrame vacío después de filtrar por meses válidos. Regresando {archivo[:19]} vacío.")
                    return df_filtrado
                    
                # Se calcula la media 
                PA_60_med_m = df_filtrado[['Valor']].resample('M').mean()

                # Cambio de contenido de columnas
                PA_60_med_m['Station'] = df['Station'].iloc[0]
                PA_60_med_m['Station'] = PA_60_med_m['Station'].astype('int64')
                PA_60_med_m['Sensor'] = 'PA_60_MEDIA_M_QC'
                
                # Reset index
                PA_60_med_m.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['Station', 'Sensor', 'Fecha', 'Valor']
                # Reordenar las columnas usando el nuevo orden
                PA_60_med_m = PA_60_med_m[nuevo_orden]
            
                # Convertir 'Fecha' de nuevo a datetime para uniformidad
                PA_60_med_m['Fecha'] = pd.to_datetime(PA_60_med_m['Fecha'])
                
                return PA_60_med_m
            
            dfmn_med_m = PA_60_MEDIA_M(dfmn_med_d_c)
            
            # Presión atmosférica media anual
            dfmn_med_m_c = dfmn_med_m.copy()
            def PA_60_MEDIA_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                if df.empty:
                    return df
                df.reset_index(inplace=True)
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                # Función para verificar si una hora específica tiene suficientes datos
                def complet_anio(sub_df):
                    return len(sub_df) >= 12 * porc_min
                   
                # Antes de resample, establecer la columna 'Fecha' como índice
                df.set_index(columna_fecha, inplace=True)
            
                # Luego de establecer el índice, aplicar resample
                df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
                # Verificar si el DataFrame filtrado está vacío
                if df_filtrado.empty:
                    print(f"DataFrame vacío después de filtrar por años válidos. Regresando {archivo[:19]} vacío.")
                    return df_filtrado
                    
                # Se calcula la media    
                PA_60_med_a = df_filtrado[['Valor']].resample('Y').mean()

                # Cambio de contenido de columnas
                PA_60_med_a['Station'] = df['Station'].iloc[0]
                PA_60_med_a['Station'] = PA_60_med_a['Station'].astype('int64')
                PA_60_med_a['Sensor'] = 'PA_60_MEDIA_A_QC'
                
                # Reset index
                PA_60_med_a.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['Station', 'Sensor', 'Fecha', 'Valor']
                # Reordenar las columnas usando el nuevo orden
                PA_60_med_a = PA_60_med_a[nuevo_orden]
            
                # Convertir 'Fecha' de nuevo a datetime para uniformidad
                PA_60_med_a['Fecha'] = pd.to_datetime(PA_60_med_a['Fecha'])
                
                return PA_60_med_a
            
            dfmn_med_a = PA_60_MEDIA_A(dfmn_med_m_c)

            if not dfmn_med_d.empty:
                dfmn_med_d.to_csv(os.path.join(carpeta, 'PA_60_MEDIA_D_QC', f'{archivo[:19]}.csv'), index=False)
            if not dfmn_med_m.empty:
                dfmn_med_m.to_csv(os.path.join(carpeta, 'PA_60_MEDIA_M_QC', f'{archivo[:19]}.csv'), date_format='%Y-%m', index=False)
            if not dfmn_med_a.empty:
                dfmn_med_a.to_csv(os.path.join(carpeta, 'PA_60_MEDIA_A_QC', f'{archivo[:19]}.csv'), date_format='%Y',index=False)

In [ ]:
calc_deriv_PA_60('ReadyToCassandraFiles_Tmin')

### Cálculo promedios horarios mensuales multianuales

In [ ]:
def calc_hmMa_patm(carpeta, chunk_size=540000):
    archivos = os.listdir(carpeta)

    # Se recorre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
        
            # Se procesan los archivos csv por fragmentos
            reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=chunk_size)
            
            for chunk in reader:
                # Se generan dataframes analizados
                # De cada chunk se transforma a datetime la serie/columna 'Fecha'
                try:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                except ValueError:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S')
                    chunk = chunk[~chunk['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO0','0PAT','0PER']]))]

                # Se hace la agrupación para cálculo de medias horarias mensuales multianuales
                hym_ma = chunk['Valor'].groupby(by =[chunk["Fecha"].dt.month, chunk["Fecha"].dt.hour]).mean().unstack(level=0)

                # Crear un archivo Excel y agregar los datos
                wb = Workbook()
                ws = wb.active
                ws.title = "Datos"
                
                # Agregar datos al archivo Excel
                for r in dataframe_to_rows(hym_ma.reset_index(), index=False, header=True):
                    ws.append(r)
                
                # Crear la gráfica de dispersión
                chart = ScatterChart()
                chart.title = "Valores Promedio por Hora y Mes"
                chart.style = 13
                chart.x_axis.title = 'Hora del día'
                chart.y_axis.title = 'Valor'
                
                # Aumentar el tamaño del gráfico
                chart.width = 20  # Anchura del gráfico (pulgadas)
                chart.height = 12  # Altura del gráfico (pulgadas)
                
                # Fijar el máximo valor del eje x
                chart.x_axis.scaling.max = 23
                chart.x_axis.scaling.min = 0
                chart.x_axis.majorUnit = 1
                
                # Agregar series a la gráfica
                colors = ['1F77B4', 'FF7F0E', '2CA02C', 'D62728', '9467BD', '8C564B', 'E377C2', '7F7F7F', 'BCBD22', '17BECF', 'AEC7E8', 'FFBB78']
                for i in range(2, 14):  # Columnas B a M (meses 1 a 12)
                    xvalues = Reference(ws, min_col=1, min_row=2, max_row=25)
                    yvalues = Reference(ws, min_col=i, min_row=1, max_row=25)
                    series = Series(yvalues, xvalues, title_from_data=True)
                    series.graphicalProperties.line.solidFill = colors[i % len(colors)]  # Asignar colores a las líneas
                    series.graphicalProperties.line.width = 30000  # Ajustar el grosor de las líneas
                    series.marker.symbol = 'circle'  # Cambiar el marcador a círculo
                    series.marker.size = 5
                    series.marker.graphicalProperties.solidFill = colors[i % len(colors)]  # Cambiar el color del marcador
                    chart.series.append(series)
                
                # Insertar la gráfica en la hoja de cálculo
                ws.add_chart(chart, "O2")

                # Nombres archivos
                nombre_archivo_salida = os.path.join(carpeta, archivo[:19] + '_hm_ma.xlsx') #archivo[:22] el de qc
                # Guardar el archivo Excel
                wb.save(nombre_archivo_salida)

In [ ]:
calc_hmMa_patm('QCResult_Patm/V3')